In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow import keras
from keras.models import Sequential

In [ ]:
def append_ext(fn):
    return fn+".png"
def hotOne(fn):
    return fn+".png"
def toString(fn):
    return str(fn)

df = pd.read_csv('../input/vcomlabels/multiclass.csv')
df_3classes = df.copy()
df_3classes["id"]= df_3classes["id"].apply(append_ext)
df_3classes["attribute_ids"] = df_3classes["attribute_ids"].apply(toString).apply(lambda x: x if x == '51' or x == '13' else '-1')

df_3classes

In [ ]:
df_lowClasses = df.copy()
df_lowClasses["id"]= df_lowClasses["id"].apply(append_ext)
df_lowClasses["attribute_ids"] = df_lowClasses["attribute_ids"].apply(toString)
df_lowClasses = df_lowClasses[ (df_lowClasses['attribute_ids']!='51') & (df_lowClasses['attribute_ids']!='13') ]
df_lowClasses

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_3classes['id'], df_3classes['attribute_ids'], stratify = df_3classes['attribute_ids'], test_size=0.15, random_state=42)

#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify = y_train, test_size=0.15, random_state=42)
#X_train,X_test

In [ ]:
frame = [X_train, y_train]
numpy_data= np.array(frame)
df_train = pd.DataFrame(data=numpy_data).T.rename(columns={0: "id", 1: "attribute_ids"})
df_train

In [ ]:
#frame = [X_val, y_val]
#numpy_data= np.array(frame)
#df_val = pd.DataFrame(data=numpy_data).T.rename(columns={0: "id", 1: "attribute_ids"})
#df_val

In [ ]:
frame = [X_test, y_test]
numpy_data= np.array(frame)
df_test = pd.DataFrame(data=numpy_data).T.rename(columns={0: "id", 1: "attribute_ids"})
df_test

In [ ]:
unique, counts = np.unique(df_train["attribute_ids"], return_counts=True)
counts = dict(zip(unique, counts))
counts

In [ ]:
def process(image):
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    return image

datagen=ImageDataGenerator(rescale=1./255, validation_split=0.2, preprocessing_function=process,
                            rotation_range=30,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                          )


train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=16,subset="training")
validation_generator=datagen.flow_from_dataframe(dataframe=df_train, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=16,subset="validation")

datagen_test=ImageDataGenerator(rescale=1./255, validation_split=0.0, preprocessing_function=process,
                            rotation_range=30,
                            width_shift_range=0,
                            height_shift_range=0,
                            zoom_range=0,
                            horizontal_flip=False,
                          )
test_generator=datagen_test.flow_from_dataframe(dataframe=df_test, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=16,subset="training")

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [ ]:
METRICS = [
      keras.metrics.CategoricalAccuracy(name='acc'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      'accuracy',
]


In [ ]:
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from keras.applications.inception_v3 import InceptionV3

In [ ]:
#model1 = InceptionV3(include_top = False, weights = "imagenet", input_shape=(224, 224, 3))

In [ ]:
"""flat1 = Flatten()(model1.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
dropout1 = Dropout(0.5)(class1)
output = Dense(3, activation='softmax')(dropout1)
# define new model
model1 = Model(inputs=model1.inputs, outputs=output)
model1.summary()"""

In [ ]:
base_model_3 = InceptionV3(weights='imagenet', 
                                include_top=False, )
base_model_3.trainable = False
add_model = Sequential()
add_model.add(base_model_3)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dense(1024, activation='relu'))
add_model.add(Dropout(0.5))
add_model.add(Dense(3, 
                    activation='softmax'))
model1 = add_model
model1.summary()

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size


model1.compile( keras.optimizers.RMSprop(learning_rate=0.0001), 
               loss="categorical_crossentropy", metrics=METRICS)
model1.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=5)

In [ ]:
base_model_3.trainable = True
model1.summary()

In [ ]:
model1.compile( keras.optimizers.RMSprop(learning_rate=0.0001), 
               loss="categorical_crossentropy", metrics=METRICS)
model1.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=10)

In [ ]:
test_generator.reset()
evaluate = model1.evaluate(test_generator)
evaluate

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
test_generator.reset()
Y_pred = model1.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

In [ ]:
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names = test_generator.class_indices.keys()))

In [ ]:
unique, counts = np.unique(df_lowClasses["attribute_ids"], return_counts=True)
counts = dict(zip(unique, counts))
counts

In [ ]:
len(counts)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_lowClasses['id'], df_lowClasses['attribute_ids'], stratify = df_lowClasses['attribute_ids'], test_size=0.15, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify = y_train, test_size=0.15, random_state=42)
#X_train,X_test

In [ ]:
frame = [X_train, y_train]
numpy_data= np.array(frame)
df_train = pd.DataFrame(data=numpy_data).T.rename(columns={0: "id", 1: "attribute_ids"})
df_train

In [ ]:
unique, counts = np.unique(df_train["attribute_ids"], return_counts=True)
counts = dict(zip(unique, counts))
max_key = max(counts, key=counts.get)
max_value = counts[max_key]
target_count = {}
for key in counts.keys():
    val = counts[key]
    if max_value / val < 2:
        target_count[key] = val
    elif max_value / val < 4:
        target_count[key] = val * 1.5
    elif max_value / val < 8:
        target_count[key] = val * 3.5
    elif max_value / val < 14:
        target_count[key] = val * 6
    elif max_value / val < 22:
        target_count[key] = val * 9
    elif max_value / val < 32:
        target_count[key] = val * 11
    else:
        target_count[key] = val * 13
    target_count[key] = int(target_count[key])

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42,sampling_strategy=target_count)
X = np.array(df_train["id"]).reshape(-1, 1)
y = np.array(df_train["attribute_ids"]).reshape(-1, 1)
X_res, y_res = ros.fit_resample(X, y)
X_res, y_res

In [ ]:
frame = [X_res[:,0], y_res]
numpy_data= np.array(frame)
df_train = pd.DataFrame(data=numpy_data).T.rename(columns={0: "id", 1: "attribute_ids"})
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

In [ ]:
frame = [X_val, y_val]
numpy_data= np.array(frame)
df_val = pd.DataFrame(data=numpy_data).T.rename(columns={0: "id", 1: "attribute_ids"})
df_val

In [ ]:
frame = [X_test, y_test]
numpy_data= np.array(frame)
df_test = pd.DataFrame(data=numpy_data).T.rename(columns={0: "id", 1: "attribute_ids"})
df_test

In [ ]:
def process(image):
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    return image

datagen=ImageDataGenerator(rescale=1./255, validation_split=0.0, preprocessing_function=process,
                            rotation_range=30,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                          )

train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=32,subset="training")
validation_generator=datagen.flow_from_dataframe(dataframe=df_val, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=16,subset="training")

datagen_test=ImageDataGenerator(rescale=1./255, validation_split=0.0, preprocessing_function=process,
                            rotation_range=30,
                            width_shift_range=0,
                            height_shift_range=0,
                            zoom_range=0,
                            horizontal_flip=False,
                          )
test_generator=datagen_test.flow_from_dataframe(dataframe=df_test, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=16,subset="training")

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [ ]:
"""
model2 = InceptionV3(include_top = False, weights = "imagenet", input_shape=(224, 224, 3))
for layer in model2.layers:
    layer.trainable = False
    
flat1 = Flatten()(model2.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
dropout1 = Dropout(0.5)(class1)
output = Dense(48, activation='softmax')(dropout1)
# define new model
model2 = Model(inputs=model2.inputs, outputs=output)

model2.summary()
"""

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(48, activation='softmax')(x)

# this is the model we will train
model2 = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model2.summary()

In [ ]:
from sklearn.utils import class_weight 
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_generator.classes), 
            train_generator.classes)

train_class_weights = dict(enumerate(class_weights))
train_class_weights

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size


model2.compile( keras.optimizers.RMSprop(learning_rate=0.0001), 
               loss="categorical_crossentropy", metrics=METRICS)
model2.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=5, class_weight = train_class_weights)

In [ ]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
mark = 249
for layer in model2.layers[:mark]:
   layer.trainable = False
for layer in model2.layers[mark:]:
   layer.trainable = True

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size


model2.compile( keras.optimizers.RMSprop(learning_rate=0.0001), 
               loss="categorical_crossentropy", metrics=METRICS)
model2.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=10, class_weight = train_class_weights)

In [ ]:
for layer in model2.layers[0:]:
   layer.trainable = True

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size


model2.compile( keras.optimizers.RMSprop(learning_rate=0.0001), 
               loss="categorical_crossentropy", metrics=METRICS)
model2.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=5, class_weight = train_class_weights)

In [ ]:
test_generator.reset()
evaluate = model2.evaluate(test_generator)
evaluate

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
test_generator.reset()
Y_pred = model2.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
Y_pred.argmax(axis=1)

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

In [ ]:
Y_pred[0]

In [ ]:
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names = test_generator.class_indices.keys()))

In [ ]:
from keras.applications.resnet_v2 import preprocess_input
from keras.applications.resnet_v2 import decode_predictions
from keras.applications.resnet_v2 import ResNet50V2

In [ ]:
def process(image):
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    return image

datagen=ImageDataGenerator(rescale=1./255, validation_split=0.0, preprocessing_function=process,
                            rotation_range=30,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                          )

train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=32,subset="training")
validation_generator=datagen.flow_from_dataframe(dataframe=df_val, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=16,subset="training")

datagen_test=ImageDataGenerator(rescale=1./255, validation_split=0.0, preprocessing_function=process,
                            rotation_range=30,
                            width_shift_range=0,
                            height_shift_range=0,
                            zoom_range=0,
                            horizontal_flip=False,
                          )
test_generator=datagen_test.flow_from_dataframe(dataframe=df_test, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=16,subset="training")

In [ ]:
from keras.layers import Concatenate
from keras.layers import BatchNormalization
base_model = ResNet50V2(weights='imagenet', include_top=False)
x0 = base_model.output
x1 = GlobalAveragePooling2D()(x0)
x2 = GlobalMaxPooling2D()(x0)
x = Concatenate()([x1,x2])

x = BatchNormalization()(x)
#     x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(48, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
base_model.trainable = False
model.summary()

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size


model.compile( keras.optimizers.RMSprop(learning_rate=0.0001), 
               loss="categorical_crossentropy", metrics=METRICS)
model.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=5, class_weight = train_class_weights)

In [ ]:
model.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=5, class_weight = train_class_weights)

In [ ]:
base_model.trainable = True
model.compile( keras.optimizers.RMSprop(learning_rate=0.0001), 
               loss="categorical_crossentropy", metrics=METRICS)
model.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=10, class_weight = train_class_weights)

In [ ]:
test_generator.reset()
evaluate = model.evaluate(test_generator)
evaluate

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
test_generator.reset()
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

In [ ]:
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names = test_generator.class_indices.keys()))

In [ ]:
from keras.applications.inception_v3 import preprocess_input
from keras.applications.inception_v3 import decode_predictions
from keras.applications.inception_v3 import InceptionV3

In [ ]:
def process(image):
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    return image

datagen=ImageDataGenerator(rescale=1./255, validation_split=0.0, preprocessing_function=process,
                            rotation_range=30,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                          )

train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=32,subset="training")
validation_generator=datagen.flow_from_dataframe(dataframe=df_val, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=16,subset="training")

datagen_test=ImageDataGenerator(rescale=1./255, validation_split=0.0, preprocessing_function=process,
                            rotation_range=30,
                            width_shift_range=0,
                            height_shift_range=0,
                            zoom_range=0,
                            horizontal_flip=False,
                          )
test_generator=datagen_test.flow_from_dataframe(dataframe=df_test, directory="../input/imet-2019-fgvc6/train",
                                            x_col="id", y_col="attribute_ids", class_mode="categorical",
                                            target_size=(224,224), batch_size=16,subset="training")

In [ ]:
from keras.layers import Concatenate
from keras.layers import BatchNormalization
base_model4 = InceptionV3(weights='imagenet', include_top=False)
x0 = base_model4.output
x1 = GlobalAveragePooling2D()(x0)
x2 = GlobalMaxPooling2D()(x0)
x = Concatenate()([x1,x2])

x = BatchNormalization()(x)
#     x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(48, activation='sigmoid')(x)

# this is the model we will train
model4 = Model(inputs=base_model4.input, outputs=predictions)
base_model4.trainable = False

In [ ]:
from sklearn.utils import class_weight 
class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_generator.classes), 
            train_generator.classes)

train_class_weights = dict(enumerate(class_weights))
train_class_weights

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size


model4.compile( keras.optimizers.RMSprop(learning_rate=0.0001), 
               loss="categorical_crossentropy", metrics=METRICS)
model4.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=5, class_weight = train_class_weights)

In [ ]:
model4.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=5, class_weight = train_class_weights)

In [ ]:
mark = 249
for layer in model4.layers[:mark]:
   layer.trainable = False
for layer in model4.layers[mark:]:
   layer.trainable = True

In [ ]:
model4.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=5, class_weight = train_class_weights)

In [ ]:
for layer in model4.layers[0:]:
   layer.trainable = True

In [ ]:
model4.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN,
           validation_data=validation_generator, validation_steps=STEP_SIZE_VALID,
           callbacks=[early_stopping],epochs=5, class_weight = train_class_weights)